<a href="https://colab.research.google.com/github/reidnclark/MFDPRepo/blob/main/mean_reversion_trading_algorithm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multipurpose Financial Data Processor (MFDP)
---

### **1) Fundamental Analysis**

##### **Test Long Stocks** (Buy & Hold):
---
- ##### Canadian Natural Resources Limited - TOR:CNQ.TO
- ##### Nutrien Ltd. - NYQ:NTR


- ##### Shin-Etsu Chemical, Co., Ltd. - PNK:SHECY
- ##### Siemens Energy AG - PNK:SMEGF
- ##### Taiwan Semiconductor, Mfg. Co., Ltd. - NYQ:TSM
- ##### Dollarama Inc. - TOR:DOL.TO


---

#### Full Code:

##### Input Arguments:

##### Import Libraries:

In [168]:
import yfinance as yf
import pandas as pd
from babel.numbers import format_currency, format_percent

# INPUTS

In [169]:
ticker_input = 'NTR'
test_ticker_assert = yf.Ticker(ticker_input)

financial_attributes_input = ['EBITDA'] # just use one for forecaster
cashflow_attributes_input = ['Free Cash Flow'] # ditto above
balsheet_attributes_input = ['Total Debt']
basic_attributes_input = ['beta', 'marketCap', 'sharesOutstanding']

# MASTER SEND DCF MODEL

#### Get Data Function:

In [170]:
def get_basic_data(attributes_data, attributes_input):
    basic_data_dict = {}
    for attr in attributes_input:
      basic_data_dict[attr] = attributes_data.get(attr)
    return basic_data_dict

def get_attr_hist_data(attributes_data, attributes_input):
    for attr in attributes_input:
      attr_hist_data = attributes_data.loc[attr].dropna()
    return attr_hist_data

def forecast(attr_hist_data):
    initial_value = attr_hist_data.iloc[-1]
    ending_value = attr_hist_data.iloc[0]
    #compound annual growth rate
    cagr = ((ending_value / initial_value)**(1/len(attr_hist_data))) - 1
    print(f'Compounded annual {attr_hist_data.name} growth rate: {(cagr*100):.2f}%')
    forecasted_list = []
    for i in range(len(attr_hist_data)):
      forecast_value = ending_value * ((1+cagr)**(i+1))
      forecasted_list.append(forecast_value)
    return forecasted_list

def get_data(ticker_input: tuple,
             financial_attributes_input: list[str],
             cashflow_attributes_input: list[str],
             balsheet_attributes_input: list[str],
             basic_attributes_input: list[str]):

  yf_ticker = yf.Ticker(ticker_input)

  # access yfinance financial attributes (For this code, we just need the 'EBITDA' metric)
  # EBITDA = Earnings before Interest, Tax, Deductions & Amortization (Loan Payments)
  financial_attributes = yf_ticker.financials
  financial_attributes_hist_data = get_attr_hist_data(financial_attributes, financial_attributes_input)
  forecasted_list_financials = forecast(financial_attributes_hist_data)

  # access yfinance data from company's cash flow statement (we just need the 'Free Cash Flow' metric)
  cashflow_attributes = yf_ticker.cashflow
  cashflow_attributes_hist_data = get_attr_hist_data(cashflow_attributes, cashflow_attributes_input)
  forecasted_list_cashflow = forecast(cashflow_attributes_hist_data)

  # access yfinance balance sheet data for company (we just need 'Total Debt')
  balsheet_attributes = yf_ticker.balance_sheet
  balsheet_attributes_hist_data = get_attr_hist_data(balsheet_attributes, balsheet_attributes_input).iloc[0]

  # basic summary information for company (we need the 'beta', 'marketCap' and 'sharesOutstanding' )
  basic_attributes = yf_ticker.info
  basic_attributes_hist_data = get_basic_data(basic_attributes, basic_attributes_input)

  return forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker

forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker = get_data(ticker_input,
                                                                           financial_attributes_input,
                                                                           cashflow_attributes_input,
                                                                           balsheet_attributes_input,
                                                                           basic_attributes_input
                                                                                      )

# data dict
ticker = yf_ticker

total_cash = (yf_ticker.balance_sheet.loc['Cash And Cash Equivalents'].iloc[0])
total_equity = (test_ticker_assert.balance_sheet).loc['Stockholders Equity'].iloc[0]

total_debt = balsheet_attributes_hist_data
interest_expenses = (yf_ticker.income_stmt.loc['Interest Expense'].iloc[0])
cost_of_debt = interest_expenses / total_debt

total_shares = basic_attributes_hist_data['sharesOutstanding']

beta = basic_attributes_hist_data['beta']
market_cap = basic_attributes_hist_data['marketCap']
ticker_name = ticker.info.get('longName')

Compounded annual EBITDA growth rate: 14.00%
Compounded annual Free Cash Flow growth rate: 7.79%


#### Compute the Expected Return ( *E ( x )*  ) of an investment given:
---

##### 1. Risk-free interest Rate ( **R$_f$** ), the % Yield on a 1-yr Government Bond (Canada).
##### 2. Beta ( ***&beta;*** ), the price volatility compared to the S&P 500.
##### 3. Expected Market Return ( **R$_m$** ), the % 1-yr return of the S&P 500.




##### Use **CAPM** Formula Function:
---
#### Capital Asset Pricing Model
---
###### Contributions: *Markowitz, H.* (1952), *Sharpe, W.* (1964)
###### Variations: Fama-French (1992), Carhart (1997)

### $$ E(R_i) = R_f + \beta_i (E(R_m) - R_f) $$

In [171]:
r_f = 4.75 / 100 # risk-free rate ( return subscript f, % )
b_i = beta # beta ( beta subscript i, ration )
e__r_m = 0.10 # expected market return ( e(return subscript m, %) )

e__r_i = r_f + b_i*(e__r_m - r_f) # expected return on investment
print(f"Expected Return of {ticker.info.get('longName')}: {(e__r_i*100):.2f}%")

Expected Return of Nutrien Ltd.: 9.76%


---
##### **WACC** Formula Function:
---
#### Weighted Average Cost of Capital
---
###### Contributions: *Modigliani, F. & Miller, M. (1958)*

### $$ WACC = \frac{E}{V} \cdot r_e + \frac{D}{V} \cdot r_d \cdot (1 - T) $$

In [172]:
E = total_equity
D = total_debt
R_e = e__r_i # capm result
R_d = cost_of_debt

T = 0.25 # tax rate

wacc = (E / (E+D))*R_e + (D / (E+D))*R_d*(1-T) # weighted average cost of capital
wacc
print(f"Minimum Break-Even Return for {ticker.info.get('longName')}: {(wacc*100):.2f}%")
# "to cover cost of capital expenditures"


Minimum Break-Even Return for Nutrien Ltd.: 8.03%


#### **Terminal Value (TV)** Formula Function:

---

In [173]:
fcf_n = forecasted_list_cashflow[-1] # value of final year in fcf forecast
g = 0.03 # perpetual growth rate (can use GDP growth estimate per year, (3 %), or ind.)

tv = (fcf_n * (1+g)) / (wacc - g)

def format_dollar(amount):
    """Format an integer as a dollar amount."""
    return f"${amount:,.2f}"

print(f"Estimated Intrinsic Value, {ticker.info.get('longName')}: {format_dollar(tv)}")
print(f"Current Value, {ticker.info.get('longName')}: {format_dollar(market_cap)}")
# in later version, add in tv financial metric multiple to find average. for now can just use perp.

Estimated Intrinsic Value, Nutrien Ltd.: $66,253,359,072.12
Current Value, Nutrien Ltd.: $24,316,037,120.00


#### **Discount** Function and Ent, Eq:

---

In [174]:
#print(format_dollar(i) for i in forecasted_list_cashflow)

pv_of_fcfs = []
discount_factors = []

for i in range(len(forecasted_list_cashflow)):
  discount_factor = (1 / ((1+wacc)**(i+1)))
  discount_factors.append(discount_factor)
  fcf_at_i = discount_factor * forecasted_list_cashflow[i]
  pv_of_fcfs.append(fcf_at_i)

pv_of_tv = discount_factors[-1]*tv

enterprise_value = (sum(pv_of_fcfs)) + pv_of_tv

equity_value = enterprise_value - total_debt + total_cash

share_price = equity_value / total_shares

#### **Resulting Share Price**:

---

# RESULTS

In [175]:
intrinsic_price = share_price

print(f'{ticker_name} Intrinsic Price: ${intrinsic_price:.2f}')
print('---------------------------------------------------------')
current_price = yf_ticker.history(period='1d')['Close'].iloc[0]
print(f'{ticker_name} Current Price: ${current_price:.2f}')
print('---------------------------------------------------------')

difference = 1 - (current_price / intrinsic_price)
print(f'Margin of Difference: {difference*100:.2f}%')

print('---------------------------------------------------------')
if current_price < intrinsic_price:
  print('BUY')
else:
  print('SHORT')

Nutrien Ltd. Intrinsic Price: $94.09
---------------------------------------------------------
Nutrien Ltd. Current Price: $49.03
---------------------------------------------------------
Margin of Difference: 47.89%
---------------------------------------------------------
BUY
